In [1]:
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType,StringType,DoubleType,DecimalType
from pyspark.sql.functions import * 

from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext

sc = SparkContext.getOrCreate()
gc = GlueContext(sc)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
11,application_1642400897224_0012,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:

# Create dynamic frame
source_df = gc.create_data_frame_from_catalog( 
    database = "trucapedb", 
    table_name = "trucape_invoices", 
)

# source_dyf = gc.create_data_frame.from_options(
#     connection_type='s3',
#     connection_options={'paths': ["s3://sol-dev-source/Trucape-Invoices"]},
#     format='csv,'
# )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
len(source_df.columns)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

9

In [ ]:
source_df.show()

In [ ]:
source_df.printSchema()

In [14]:
# ADD YEAR TO DUE DATE
# source_df_with_date_format = source_df.withColumn("due date", F.concat(col("due date"),F.lit("-2022"))) \
#     .show()

#  ADD YEAR TO ISSUE DATE
# source_df_with_date_format = source_df.withColumn("issue date", F.concat(col("issue date"),F.lit("-2022"))) \
#     .show()

# add year to columns: due date & isue date 
# due_date="due date"
# issue_date="issue date" - create variable for long functions

# Add year to date columns
source_df = source_df.withColumn("due date", F.concat(col("due date"),F.lit("-2022"))) \
    .withColumn("issue date", F.concat(col("issue date"),F.lit("-2022"))) 
    


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
source_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------+--------------------+------------+--------+-------------+------------+-----------+----------+
|external reference|number|        counterparty|  amount due|currency|cost currency|costing rate|   due date|issue date|
+------------------+------+--------------------+------------+--------+-------------+------------+-----------+----------+
|          SC25505A|137113|   Fruit Traders LLC|  10,800.00 |     USD|          ZAR|      15.862|24-Jan-2022|3-Jan-2022|
|          SC25506A|137114|   Fruit Traders LLC|  10,800.00 |     USD|          ZAR|      15.862|24-Jan-2022|3-Jan-2022|
|          SC25503A|137116|   Fruit Traders LLC|  10,800.00 |     USD|          ZAR|      15.862|24-Jan-2022|3-Jan-2022|
|          SC25504A|137117|   Fruit Traders LLC|  10,800.00 |     USD|          ZAR|      15.862|24-Jan-2022|3-Jan-2022|
|          SC25499A|137078|   Fruit Traders LLC|  10,134.00 |     USD|          ZAR|      15.862|24-Jan-2022|3-Jan-2022|
|          SC25500A|137079|   Fr

In [16]:
# DUE DATE & ISSUE DATE STRING TO DATE TYPE

# .SELECT - TUPLE ISSUE
source_df = source_df.select(col("due date"),to_date(F.col("due date"), "d-MMM-yyy"), \
                             col("issue date"),to_date(F.col("issue date"), "d-MMM-yyy")) \
                       
# RETURNS NULL TYPE IN COLUMNS
# source_df = source_df.withColumn("due date",col("due date").cast(DateType())) \
#             .withColumn("issue date", col("issue date").cast(DateType())) \

# TIME STAMP - GIVES NULL
# source_df = source_df.select(
#     F.to_timestamp(F.col('due date'), 'yyyy-MM-dd')
# )

# FORMAT COLUMNS: DUE DATE & ISSUE DATE 
# due_date_col = col("due date")
# issue_date_col = col("issue date")
# due_date = to_date(F.due_date_col, "d-MMM-yyy")
# issue_date = to_date(F.issue_date_col, "d-MMM-yyy")

# source_df = source_df.select(col("due date"),to_date(F.col("due date"), "d-MMM-yyy")) 

# TIMESTAMP TYPE TO STRING - NO ATTRIBUTE CAST
# source_ = source_df.withColumn(
#          to_timestamp(col(('due date').cast('string')))) 

# SPARK TIMESTAMP
# source_df = source_df.select(to_timestamp(lit('due date'),'yyyy-MM-dd')) - GIVES NULL
# source_df = source_df.withColumn('due date', F.to_timestamp('due date', 'yyyy-MM-dd')) - GIVES ERROR



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
source_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------------------+----------+----------------------------------+
|   due date|to_date(`due date`, 'd-MMM-yyy')|issue date|to_date(`issue date`, 'd-MMM-yyy')|
+-----------+--------------------------------+----------+----------------------------------+
|24-Jan-2022|                      2022-01-24|3-Jan-2022|                        2022-01-03|
|24-Jan-2022|                      2022-01-24|3-Jan-2022|                        2022-01-03|
|24-Jan-2022|                      2022-01-24|3-Jan-2022|                        2022-01-03|
|24-Jan-2022|                      2022-01-24|3-Jan-2022|                        2022-01-03|
|24-Jan-2022|                      2022-01-24|3-Jan-2022|                        2022-01-03|
|24-Jan-2022|                      2022-01-24|3-Jan-2022|                        2022-01-03|
|24-Jan-2022|                      2022-01-24|3-Jan-2022|                        2022-01-03|
| 2-Feb-2022|                      2022-02-02|3-Jan-2022|             